In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sampleSubmission.csv', 'test.csv']


In [2]:
train = pd.read_csv("../input/train.csv")

In [3]:
test = pd.read_csv("../input/test.csv")

In [4]:
train.head(10)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,1,1
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,0,2
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1,2,3
8,2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000,1,7,8
9,2011-01-01 09:00:00,1,0,0,1,13.12,17.425,76,0.0000,8,6,14


In [5]:
#결측치 확인
train.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64

In [6]:
train.dtypes
#datetime 형식이 object이다 , datetime형식이 아니다

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object

In [7]:
#datetime 바꾸는 형식
train = pd.read_csv("../input/train.csv", parse_dates = ["datetime"])
test = pd.read_csv("../input/test.csv", parse_dates = ["datetime"])

In [8]:
train.dtypes

datetime      datetime64[ns]
season                 int64
holiday                int64
workingday             int64
weather                int64
temp                 float64
atemp                float64
humidity               int64
windspeed            float64
casual                 int64
registered             int64
count                  int64
dtype: object

In [9]:
# datetime 바꾸는 형식
train["datetime"] = train["datetime"].astype("datetime64")

In [10]:
#새로운 칼럼을 만듬
train["year"] = train["datetime"].dt.year
train["hour"] = train["datetime"].dt.hour
train["dayofweek"] = train["datetime"].dt.dayofweek

test["year"] = test["datetime"].dt.year
test["hour"] = test["datetime"].dt.hour
test["dayofweek"] = test["datetime"].dt.dayofweek

In [11]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,hour,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,2,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,3,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,4,5


In [12]:
# 정답 변수인 종속 변수를 따로 분리해줘야한다.
y_train = train["count"]

In [13]:
# 중요하지 않은 칼럼을 제거한다. train set , test set 열의 숫자가 반드시 똑같아야한다. Test set에 정답변수가 없다.
train = train.drop(["datetime", "windspeed", "casual", "registered", "count"], 1)


In [14]:
test = test.drop(["datetime", "windspeed"], 1)

In [15]:
test.head()

,season,holiday,workingday,weather,temp,atemp,humidity,year,hour,dayofweek
0,1,0,1,1,10.66,11.365,56,2011,0,3
1,1,0,1,1,10.66,13.635,56,2011,1,3
2,1,0,1,1,10.66,13.635,56,2011,2,3
3,1,0,1,1,10.66,12.880,56,2011,3,3
4,1,0,1,1,10.66,12.880,56,2011,4,3


In [16]:
train.head()

,season,holiday,workingday,weather,temp,atemp,humidity,year,hour,dayofweek
0,1,0,0,1,9.84,14.395,81,2011,0,5
1,1,0,0,1,9.02,13.635,80,2011,1,5
2,1,0,0,1,9.02,13.635,80,2011,2,5
3,1,0,0,1,9.84,14.395,75,2011,3,5
4,1,0,0,1,9.84,14.395,75,2011,4,5


In [17]:
# 종속에 log 취하면 1.모델 학습 시간이 빨라짐 2. 성능이 올라감
y_train = np.log1p(y_train)

In [18]:
#모델링 과정
#1. 모델을 가져온다 (회귀 -regression : 양을 예측할때, 판매량, 가격 예측) or (분류 - classification: ox)
from sklearn.ensemble import RandomForestRegressor

#2. 모델을 선언한다. 학습 파라미터: n_esimators (나무의 갯수/ 각각의 나무는 분류기 / 각각 나무 하나하나가 학습하는 애들) information gain 높은 방향, entropy가 낮아지는 방향으로 학습 
# 각각의 나무는 전체 데이터를 학습 => 따라서 학습 횟수가 100번, 100번이 기본값
rf = RandomForestRegressor(n_estimators=100)
# RandomForest는 과적합문제가 잘 생기지 않는다.
# n_estimator 너무 많으면  1) 학습시간이 오래 걸린다. 2) 처음에는 성능이 팍팍 오르다가, 사실상 모델의 성능이 잘 안오름
# RandomForest 가성비가 좋다. 베이스라인(프로토타입) 모델 잡을때 사용

#3. 모델을 학습시킨다.
rf.fit(train,y_train)

#4. 모델을 예측한다.
preds = rf.predict(test)

In [19]:
submission=pd.read_csv("../input/sampleSubmission.csv")

In [20]:
submission.head()

,datetime,count
0,2011-01-20 00:00:00,0
1,2011-01-20 01:00:00,0
2,2011-01-20 02:00:00,0
3,2011-01-20 03:00:00,0
4,2011-01-20 04:00:00,0


In [21]:
submission["count"] = np.expm1(preds)

In [22]:
submission.head()

,datetime,count
0,2011-01-20 00:00:00,11.080358
1,2011-01-20 01:00:00,4.442765
2,2011-01-20 02:00:00,2.573367
3,2011-01-20 03:00:00,3.801308
4,2011-01-20 04:00:00,2.923830


In [23]:
submission.to_csv("allrf.csv", index=False)